# Check GPU availability
Check GPU drivers and CUDA availability for training.

In [ ]:
# Check GPU status
!nvidia-smi

# Install required packages
Install required packages for downloading dataset and training YOLOv11 model.

In [ ]:
# Install required packages
print("Installing required packages...")
%pip install -q ultralytics roboflow comet_ml
print("Packages installed successfully!")

In [ ]:
# Import necessary libraries
from google.colab import userdata
import comet_ml

# Roboflow configuration
ROBOFLOW_API_KEY=userdata.get('ROBOFLOW_API_KEY')
ROBOFLOW_WORKSPACE=userdata.get('ROBOFLOW_WORKSPACE')
ROBOFLOW_PROJECT=userdata.get('ROBOFLOW_PROJECT')
ROBOFLOW_VERSION=userdata.get('ROBOFLOW_VERSION')

# Comet ML configuration and initialization
COMET_API_KEY=userdata.get('COMET_API_KEY')

print("Initializing Comet ML...")
comet_ml.login(api_key=COMET_API_KEY, project_name="yolo11-instance-segmentation")
print("✓ Comet ML initialized successfully!")
print("Dashboard URL will be available when training starts.")

# Download dataset from Roboflow
Download dataset from Roboflow for YOLOv11 training.

In [ ]:
# Download dataset
from roboflow import Roboflow

print("Downloading dataset from Roboflow...")
print(f"Workspace: {ROBOFLOW_WORKSPACE}")
print(f"Project: {ROBOFLOW_PROJECT}")
print(f"Version: {ROBOFLOW_VERSION}")

rf = Roboflow(api_key=ROBOFLOW_API_KEY)
project = rf.workspace(ROBOFLOW_WORKSPACE).project(ROBOFLOW_PROJECT)
version = project.version(int(ROBOFLOW_VERSION))
dataset = version.download("yolov11")

print("Dataset downloaded successfully!")

# Configure TensorBoard (DEPRECATED - Using Comet ML)

**NOTE**: TensorBoard code has been replaced with Comet ML for better visualization and collaboration.
The code below is kept for reference only.

In [ ]:
# DEPRECATED: Replaced with Comet ML logging
# TensorBoard code is kept below for reference

# # Configure TensorBoard for logging
# from ultralytics import settings

# # Enable TensorBoard in Ultralytics
# settings.update({'tensorboard': True})

# # Load TensorBoard extension in Colab
# %load_ext tensorboard

# # Start TensorBoard (will monitor runs/detect directory)
# %tensorboard --logdir runs/detect

# print("TensorBoard is ready! Check the output above to view training metrics.")

print("TensorBoard logging has been replaced with Comet ML.")
print("Comet ML provides better visualization, cloud-based dashboard, and easier collaboration.")
print("The dashboard URL will appear when you start training.")


# Train YOLO11n-seg Model with Comet ML Logging

Fine-tune YOLO11n-seg (Instance Segmentation) on custom dataset with optimized parameters for Tesla T4.
Comet ML will automatically log all metrics, hyperparameters, and visualizations.

**Note**: Instance segmentation models provide both bounding boxes AND pixel-level masks for each detected object.

In [ ]:
from ultralytics import YOLO
import os

# Get the data.yaml path from the downloaded dataset
data_yaml = f"{dataset.location}/data.yaml"
print(f"Dataset location: {data_yaml}")

# Load pretrained YOLO11n-seg model (Instance Segmentation)
# This model provides both bounding boxes AND pixel-level masks
model = YOLO('yolo11n-seg.pt')
print("YOLO11n-seg model loaded successfully!")
print(f"Model type: Instance Segmentation")
print(f"Model size: {sum(p.numel() for p in model.model.parameters()) / 1e6:.2f}M parameters")

# Train the segmentation model with optimized parameters for Tesla T4
# NOTE: Comet ML will automatically log all metrics, hyperparameters, and visualizations
# Dashboard URL will appear below when training starts
print("\nStarting segmentation training with Comet ML logging...")
print("Comet dashboard URL will appear below...\n")

results = model.train(
    # Dataset configuration
    data=data_yaml,                     # Path to dataset YAML (must have segmentation annotations)

    # Training parameters
    epochs=50,                          # Number of epochs
    imgsz=640,                          # Image size (640x640)
    batch=16,                           # Batch size (optimized for T4 16GB)

    # Optimization
    device=0,                           # Use GPU
    workers=4,                          # Number of dataloader workers

    # Learning rate
    lr0=0.01,                           # Initial learning rate
    lrf=0.01,                           # Final learning rate factor

    # Augmentation (YOLO11-seg optimized)
    hsv_h=0.015,                        # HSV-Hue augmentation
    hsv_s=0.7,                          # HSV-Saturation
    hsv_v=0.4,                          # HSV-Value
    degrees=0.0,                        # Rotation
    translate=0.1,                      # Translation
    scale=0.5,                          # Scale
    fliplr=0.5,                         # Horizontal flip probability
    mosaic=1.0,                         # Mosaic augmentation
    copy_paste=0.1,                     # Copy-paste augmentation (good for segmentation)

    # Saving and logging
    save=True,                          # Save checkpoints
    save_period=10,                     # Save every 10 epochs
    project='runs/segment',             # Output directory for segmentation
    name='yolo11n_seg_product_labels',  # Experiment name
    exist_ok=True,                      # Overwrite if exists

    # Validation
    val=True,                           # Run validation after each epoch

    # Visualization
    plots=True,                         # Generate plots (including mask visualizations)

    # Early stopping
    patience=10,                        # Stop if no improvement after 50 epochs

    # Mixed precision training (speeds up training on T4)
    amp=True,                           # Automatic Mixed Precision

    # Segmentation specific
    mask_ratio=4,                       # Mask downsample ratio (default=4)
    overlap_mask=True,                  # Masks overlap during training (good for dense objects)

    # Verbose output
    verbose=True                        # Detailed output
)

print("\nTraining completed!")
print("✓ All metrics, visualizations, and model artifacts logged to Comet ML dashboard")
print(f"Best model saved at: runs/segment/yolo11n_seg_product_labels/weights/best.pt")
print(f"Last model saved at: runs/segment/yolo11n_seg_product_labels/weights/last.pt")

# Model Validation (Segmentation)

Evaluate segmentation model performance on validation set.
Metrics include both bounding box (box) and mask (seg) metrics.

In [ ]:
# Load the best trained segmentation model
best_model = YOLO('runs/segment/yolo11n_seg_product_labels/weights/best.pt')

# Validate the model
print("Running segmentation validation...")
metrics = best_model.val(data=data_yaml)

# Print detailed metrics for both Box and Mask
print("\n" + "="*70)
print("SEGMENTATION VALIDATION METRICS")
print("="*70)

# Bounding Box Metrics
print("\nBOUNDING BOX METRICS:")
print("-"*40)
print(f"mAP50-95 (Box): {metrics.box.map:.4f}")      # mAP at IoU 0.5:0.95
print(f"mAP50 (Box):    {metrics.box.map50:.4f}")    # mAP at IoU 0.5
print(f"mAP75 (Box):    {metrics.box.map75:.4f}")    # mAP at IoU 0.75
print(f"Precision (Box): {metrics.box.mp:.4f}")      # Mean precision
print(f"Recall (Box):    {metrics.box.mr:.4f}")      # Mean recall

# Segmentation Mask Metrics
print("\nSEGMENTATION MASK METRICS:")
print("-"*40)
print(f"mAP50-95 (Mask): {metrics.seg.map:.4f}")     # mAP at IoU 0.5:0.95 for masks
print(f"mAP50 (Mask):    {metrics.seg.map50:.4f}")   # mAP at IoU 0.5 for masks
print(f"mAP75 (Mask):    {metrics.seg.map75:.4f}")   # mAP at IoU 0.75 for masks
print(f"Precision (Mask): {metrics.seg.mp:.4f}")     # Mean precision for masks
print(f"Recall (Mask):    {metrics.seg.mr:.4f}")     # Mean recall for masks

print("\n" + "="*70)

# Print per-class metrics if available
if hasattr(metrics.box, 'maps') and hasattr(metrics.seg, 'maps'):
    print("\nPer-class Metrics:")
    print(f"{'Class':<15} {'Box mAP50-95':<15} {'Mask mAP50-95':<15}")
    print("-"*45)
    for i, (box_map, seg_map) in enumerate(zip(metrics.box.maps, metrics.seg.maps)):
        print(f"Class {i:<9} {box_map:<15.4f} {seg_map:<15.4f}")

print("\n✓ Validation completed!")

# Export Segmentation Model to ONNX

Export the segmentation model to ONNX format for flexible deployment.
The exported model includes both detection head and segmentation head.

In [ ]:
# Export the best segmentation model to ONNX format
print("Exporting segmentation model to ONNX format...")
print("This will include both detection and segmentation heads.")

onnx_path = best_model.export(
    format='onnx',          # ONNX format
    imgsz=640,              # Input size (640x640)
    dynamic=False,          # Fixed input size (better for deployment)
    simplify=True,          # Optimize the ONNX graph
    opset=12,               # ONNX opset version (widely compatible)
)

print(f"\n✓ ONNX export completed!")
print(f"ONNX model saved at: {onnx_path}")
print(f"Model is optimized for 640x640 input images")
print(f"Model type: Instance Segmentation (detection + mask output)")

# Verify the exported ONNX model
print("\nVerifying ONNX segmentation model...")
onnx_model = YOLO(onnx_path)
print("✓ ONNX segmentation model loaded successfully and ready for inference!")

# Test Segmentation Inference (Optional)

Test the segmentation model on sample images from validation set.
Results include both bounding boxes and segmentation masks.

In [ ]:
import glob
import numpy as np
from IPython.display import Image, display

# Get a sample image from the validation set
val_images = glob.glob(f"{dataset.location}/valid/images/*.jpg") + \
             glob.glob(f"{dataset.location}/valid/images/*.png")

if val_images:
    # Use the first image as test
    test_image = val_images[0]
    print(f"Testing segmentation on: {test_image}")

    # Run segmentation inference with the PyTorch model
    print("\nTesting with PyTorch segmentation model...")
    results_pt = best_model(test_image, conf=0.25)

    # Save and display results (includes segmentation masks)
    results_pt[0].save('inference_pytorch_seg.jpg')
    print("✓ PyTorch segmentation inference completed")

    # Run inference with the ONNX model
    print("\nTesting with ONNX segmentation model...")
    results_onnx = onnx_model(test_image, conf=0.25)
    results_onnx[0].save('inference_onnx_seg.jpg')
    print("✓ ONNX segmentation inference completed")

    # Display segmentation results
    print("\n" + "="*60)
    print("SEGMENTATION RESULTS")
    print("="*60)

    result = results_pt[0]

    # Display detection info
    print(f"\nDetected {len(result.boxes)} objects:")
    for i, box in enumerate(result.boxes):
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        print(f"  {i+1}. Class: {result.names[cls_id]}, Confidence: {conf:.2f}")

    # Display mask info if available
    if result.masks is not None:
        print(f"\nSegmentation Masks:")
        print(f"  - Number of masks: {len(result.masks)}")
        print(f"  - Mask shape: {result.masks.data.shape}")

        # Access mask data in different formats
        masks_data = result.masks.data      # Mask in matrix format (num_objects x H x W)
        masks_xy = result.masks.xy          # Mask in polygon format (list of xy coordinates)
        masks_xyn = result.masks.xyn        # Normalized polygon coordinates

        print(f"  - Available formats: matrix, polygon (xy), normalized (xyn)")

        # Calculate mask coverage
        for j, mask in enumerate(masks_data):
            mask_area = mask.sum().item()
            total_area = mask.shape[0] * mask.shape[1]
            coverage = (mask_area / total_area) * 100
            print(f"  - Mask {j+1} coverage: {coverage:.2f}% of image")
    else:
        print("\nNo segmentation masks detected")

    print("\n" + "="*60)

    # Display the image with segmentation
    print("\nResult image with segmentation masks:")
    display(Image('inference_pytorch_seg.jpg'))

else:
    print("No validation images found. Please check the dataset path.")

# Save Segmentation Models (Optional)

Download segmentation models to local machine or save to Google Drive.
Includes both PyTorch (.pt) and ONNX formats.

In [ ]:
# Save segmentation models - Upload to Google Drive
from google.colab import drive
import shutil
import os

# Check if models exist before copying
pytorch_model_path = 'runs/segment/yolo11n_seg_product_labels/weights/best.pt'

if not os.path.exists(pytorch_model_path):
    print("Error: Trained segmentation model not found.")
    print("Please run the training cell first.")
elif 'onnx_path' not in globals():
    print("Error: ONNX model not exported.")
    print("Please run the ONNX export cell first.")
else:
    # Create a temporary directory for models
    models_dir = 'trained_models_seg'
    os.makedirs(models_dir, exist_ok=True)

    # Copy PyTorch segmentation model
    print("Copying PyTorch segmentation model...")
    shutil.copy(pytorch_model_path, f'{models_dir}/yolo11n_seg_best.pt')

    # Copy ONNX segmentation model
    print("Copying ONNX segmentation model...")
    shutil.copy(onnx_path, f'{models_dir}/yolo11n_seg_best.onnx')

    # Create zip file
    print("\nCreating zip archive...")
    zip_filename = 'yolo11n_seg_trained_models'
    shutil.make_archive(zip_filename, 'zip', models_dir)
    zip_path = f'{zip_filename}.zip'

    print("\n✓ Segmentation models packaged successfully!")
    print(f"Zip file: {zip_path}")
    print(f"Size: {os.path.getsize(zip_path) / (1024*1024):.2f} MB")

    # Mount Google Drive
    print("\nMounting Google Drive...")
    drive.mount('/content/drive')

    # Copy to Google Drive
    drive_path = '/content/drive/MyDrive/yolo11n_seg_trained_models.zip'
    print(f"\nCopying to Google Drive: {drive_path}")
    shutil.copy(zip_path, drive_path)

    print("\n" + "="*70)
    print("SUCCESS! Segmentation models uploaded to Google Drive!")
    print("="*70)
    print(f"\nFile location: MyDrive/yolo11n_seg_trained_models.zip")
    print(f"Size: {os.path.getsize(drive_path) / (1024*1024):.2f} MB")
    print(f"Model type: Instance Segmentation (YOLO11n-seg)")
    print("\nYou can now:")
    print("  1. Access the file from your Google Drive")
    print("  2. Download it from drive.google.com")
    print("  3. Share it with others if needed")
    print("="*70)

    # Show local file paths as backup
    print(f"\nLocal backup files:")
    print(f"  - {os.path.abspath(zip_path)}")
    print(f"  - {os.path.abspath(models_dir)}/yolo11n_seg_best.pt")
    print(f"  - {os.path.abspath(models_dir)}/yolo11n_seg_best.onnx")